## Summaries for SCADS Application 

Utilizing Occams to build single document summaries for SCADS Demo application use 

In [ ]:
import pandas as pd
import sqlite3
import multisum as msu
import re
from tqdm import tqdm

### Access Database

In [ ]:
# get data from sqlite file
# resources: https://datacarpentry.org/python-ecology-lesson/09-working-with-sql/index.html
con = sqlite3.connect("data_large/news (1).sqlite")

# use this to check schema details and table names
# cur = con.cursor()
# cur.execute('SELECT * FROM sqlite_master')
# cur.fetchall()

df = pd.read_sql_query("SELECT * FROM article", con)
con.close()

In [ ]:
df.shape

In [ ]:
# Note: The database does not have the text from the article... boo. Merging values based on nid together.
# This dataset has scrapped the article text from the URL in 'efs/home/strotto/MIND HTML Pre-Processing.ipynb'
MIND_w_topics=pd.read_csv("~/efs/home/pcorona_content/MIND_Train_w_Topics.csv")
MIND_w_topics.sort_values(by=['Title', '_score_'], ascending=False, inplace=True)
MIND_w_topics.drop_duplicates(subset=['Title'], keep='first', inplace=True)

In [ ]:
dff = df.merge(MIND_w_topics, how='left', left_on='title', right_on='Title')

In [ ]:
to_sum = dff[~dff['Text'].isna()].reset_index()[['nid', 'cat', 'subcat', 'Title', 'abstract', 'display_date', 'Text']]
to_sum['summary'] = ''
to_sum

### Summarize

In [ ]:
# loop through all texts to summarize 
for ele in tqdm(range(0, len(to_sum))):
    documents, doc_incidences, load_time = msu.load_corpus(to_sum.loc[ele:ele].reset_index(), 'Text')
    # preprocessing occurs here, removing starting phrases and URLs
    a = [x.text for x in documents[0].sentences if not bool(re.match("[A-Z]+( |:|-){1}", str(x.text))) and not bool(re.match('(.*)(\.com|\.edu|\.net|\.org)', str(x.text)))]
    to_sum['Text'][ele] = " ".join(a)
    documents, doc_incidences, load_time = msu.load_corpus(to_sum.loc[ele:ele].reset_index(), 'Text')

    # if only 1 sentance has been recognized, then don't allow for summarization 
    if len(documents[0].sentences) > 1:
        # summarizing with scheme "sentences", of length "300"
        build_time, extract_time, sentences, doc_titles_new, extractor, sentence_weights = msu.summarize_collect(to_sum.loc[ele:ele].reset_index(), documents, doc_incidences, 'sentances', 300, False)
        summary_text = [sentences[x].text for x in range(0, len(sentences))]
    else: 
        summary_text = 'Summary Not Available'
        
    to_sum['summary'][ele] = summary_text

to_sum.rename(columns = {'Title': 'title', 'Text':'text'}, inplace=True)
to_sum.to_csv("out/sumSCADS_v3.csv", index=False)

In [ ]:
to_sum['summary'] = [None if x == '' or x == [] or x == 'Summary Not Available' else x for x in to_sum['summary']]

### Export data back to sqllite file

In [ ]:
# keep the original data (even if it doesn't have a summary)
wdf = df.merge(to_sum[['nid', 'summary']], how='left', left_on='nid', right_on='nid')

In [ ]:
# turn array into string w/ unique delimiter 
wdf['sum'] = [" | ".join(x) if type(x) == list else None for x in wdf['summary']]
wdf = wdf[['nid', 'cat', 'subcat', 'title', 'abstract', 'display_date', 'sum']]

In [ ]:
con = sqlite3.connect("data_large/news (1).sqlite")

# Write the new DataFrame to a new SQLite table
wdf.to_sql("articlesSum", con, if_exists="replace", index=False)

con.close()